In [6]:
import tweepy
from pprint import pprint
import jsonpickle
from tweepy import Stream
from tweepy.streaming import StreamListener
import time
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [7]:
auth = tweepy.OAuthHandler('iEk2lB2S9h6qdi9bG2F3OOfVX','LHOn4yNXL5ggl193syNeYaMs2jHGlw7zwDVLFc2pjVWW9XD5LO')
auth.set_access_token('1450808706-r0tIB7jGbOeHbz5EiXkUed2HdnTyNqZFGSZINTL','UCkmKWNSK6EYvk5MTT75GJERl7MPwGOMEXAsjqFnhmlBw')

api = tweepy.API(auth)
if (not api):
    print("Authentication failed :(")
else:
    print("Authentication successfull!!! :D")

Authentication successfull!!! :D


In [8]:
df = pd.read_csv(
    'corpus.txt', 
    delimiter='\t', 
    names=['event_mention_id', 'tweet_id', 'event_instance_id', 'event_mention_trigger', 'timestamp', 'text']
)

In [9]:
df1 = {
    'tweet_id':list(),
    'text':list()
}
tweets = []
dataset = []
tweet_ids = df['tweet_id'].values.tolist()

In [10]:
for i in tqdm(range(0, len(tweet_ids), 100)):
    statuses = api.statuses_lookup(tweet_ids[i: i + 100])
    dataset.append(statuses)
    
    for idx, status in enumerate(statuses):
        tweet = df.iloc[[idx]]
        df1['tweet_id'].append(status._json['id'])
        df1['text'].append(status._json['text'])
    df_prime = pd.DataFrame(df1)
    df_prime.to_csv('fetched_tweets.csv')
    time.sleep((15 * 60) + 2)

TweepError: Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/statuses/lookup.json?id=789148558448394240%2C789148558448394240%2C789150403925377025%2C789156196259201024%2C789156196259201024%2C789164534556561409%2C789164534556561409%2C789166942074449920%2C789166942074449920%2C789168410101895168%2C789170251367800833%2C789172558239195137%2C789172558239195137%2C789174030460784640%2C789180066060132353%2C789181387261612035%2C789181609576583169%2C789185292162899972%2C789185292162899972%2C789185891939909632%2C789190044300906496%2C789190044300906496%2C789224219485753344%2C789227532981669888%2C789231832160022528%2C789240224949805056%2C789255081166118912%2C789255387383996416%2C789257769744490520%2C789257769744490520%2C789259938178609152%2C789261255211188224%2C789261255211188224%2C789277256468148224%2C789278191806537728%2C789296168584871936%2C789301189179535360%2C789304318134398976%2C789338568812470273%2C789418113775333376%2C789426263

In [ ]:
import tokenizer as tk
import re

In [ ]:
df1 = pd.DataFrame(df1)
def cleanTweet(tweet):
    s = " ".join(tk.tokenize(tweet[:-1]))
    x = re.sub(r'[^\x00-\xf3]', '', s)
    return x
df1['text'] = df1['text'].apply(cleanTweet)

In [ ]:
df_main = {
    'event_mention_id': list(), 
    'tweet_id': list(), 
    'event_instance_id': list(), 
    'event_mention_trigger': list(), 
    'timestamp': list(), 
    'text':list()
}

In [ ]:
for tweet_id in df1['tweet_id'].values:
    tweet = df[tweet_id == df['tweet_id']]
    for key in df_main.keys():
        if key != 'text':
            df_main[key].append(tweet[key].values[0])
    df_main['text'].append(df1[df1['tweet_id'] == tweet_id]['text'].values[0])

In [ ]:
df_main = pd.DataFrame(df_main)

In [ ]:
def transform(line):
    line = line.lower()
    line = line.replace("'''", "'")
    line = line.replace("isn't", "is not")
    line = line.replace("wasn't", "was not")
    line = line.replace("didn't", "did not")
    line = line.replace("don't", "do not")
    line = line.replace("dont", "do not")

    line = line.replace("doesn't", "does not")
    line = line.replace("doesnt", "does not")

    line = line.replace("won't", "will not")
    line = line.replace("wont", "will not")

    line = line.replace("wouldn't", "would not")
    line = line.replace("can't", "can not")
    line = line.replace("cannot", "can not")

    line = line.replace("hasn't", "has not")
    line = line.replace("*questioned", "questioned")
    line = line.replace("targets", "target")
    line = line.replace("=", "")
        
    return line


In [ ]:
df_main['text'] = df_main['text'].apply(transform)

In [ ]:
df_main.to_csv('FinalData.csv')